In [1]:
import numpy as np
import wandb
from collections import OrderedDict

import cv2
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from test_data.factory_test import dataset_factory
from config import cfg

import lietorch
from lietorch import SE3
from geom.losses import geodesic_loss
# from src.geom.losses import geodesic_loss

# network
# from src.model import ViTEss
from logger import Logger
from cuti import build

# DDP training
import torch.multiprocessing as mp
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

from tqdm import tqdm
import random
from datetime import datetime
import os

import torchvision.models as models

from cuti import build
from config import cfg

In [2]:
db = dataset_factory(
    ["matterport"],
    datapath="/home/kmuvcl/source/CuTi/matterport",
    subepoch=0,
    is_training=False,
    gpu = 0,
    streetlearn_interiornet_type = None,
    use_mini_dataset = False
)

Building Matterport dataset
Dataset matterport has 7996 images


In [3]:
model = build(cfg)

In [4]:
ckpt = "/home/kmuvcl/source/CuTi/checkpoint/006000.pth"

In [5]:
state_dict = OrderedDict([
        (k.replace("module.", ""), v) for (k, v) in torch.load(ckpt)['model'].items()])

In [6]:
model.load_state_dict(state_dict,False)
model = model.cuda().eval()

In [7]:

test_loader = DataLoader(
 db, batch_size=cfg.SOLVER.BATCH_SIZE, num_workers=1, shuffle=False
)

In [18]:
DEPTH_SCALE =5
predictions = {'camera': {'preds': {'tran': [], 'rot': []}, 'gts': {'tran': [], 'rot': []}}}
loss_list =[]

In [20]:
with tqdm(test_loader, unit="batch") as tepoch:
    for i_batch, item in enumerate(tepoch):
        images, poses, intrinsics, lines = [x.to("cuda") for x in item]
        Ps = SE3(poses) 
        Gs = SE3.IdentityLike(Ps)
        Ps_out = SE3(Ps.data.clone())
        with torch.no_grad():
            poses_est = model(images, lines, Gs)
        #print(poses)
        geo_loss_tr, geo_loss_rot, geo_metrics =geodesic_loss(Ps_out,poses_est,train_val='test')
        loss = 10 * geo_loss_tr + 10 * geo_loss_rot
        loss_list.append(loss)

  0%|          | 0/1333 [00:00<?, ?batch/s]/home/kmuvcl/source/CuTi/ctrlc/ctrlc.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  extra_samples = torch.tensor(extra_samples, dtype = torch.float32)
  0%|          | 1/1333 [00:00<15:29,  1.43batch/s]

GS______ [SE3: size=torch.Size([6, 2]), device=cuda:0, dtype=torch.float32]
GS[0]__ __ __  SE3: size=torch.Size([6, 2]), device=cuda:0, dtype=torch.float32
GS[0][:,jj] SE3: size=torch.Size([6, 2]), device=cuda:0, dtype=torch.float32


  0%|          | 2/1333 [00:01<12:11,  1.82batch/s]

GS______ [SE3: size=torch.Size([6, 2]), device=cuda:0, dtype=torch.float32]
GS[0]__ __ __  SE3: size=torch.Size([6, 2]), device=cuda:0, dtype=torch.float32
GS[0][:,jj] SE3: size=torch.Size([6, 2]), device=cuda:0, dtype=torch.float32


  0%|          | 3/1333 [00:01<11:18,  1.96batch/s]

...

GS______ [SE3: size=torch.Size([6, 2]), device=cuda:0, dtype=torch.float32]
GS[0]__ __ __  SE3: size=torch.Size([6, 2]), device=cuda:0, dtype=torch.float32
GS[0][:,jj] SE3: size=torch.Size([6, 2]), device=cuda:0, dtype=torch.float32


100%|█████████▉| 1332/1333 [10:13<00:00,  2.14batch/s]

GS______ [SE3: size=torch.Size([6, 2]), device=cuda:0, dtype=torch.float32]
GS[0]__ __ __  SE3: size=torch.Size([6, 2]), device=cuda:0, dtype=torch.float32
GS[0][:,jj] SE3: size=torch.Size([6, 2]), device=cuda:0, dtype=torch.float32


100%|█████████▉| 1332/1333 [10:14<00:00,  2.17batch/s]


RuntimeError: mat1 dim 1 must match mat2 dim 0

In [22]:
print(loss_list)

[tensor(23.7788, device='cuda:0'), tensor(18.3913, device='cuda:0'), tensor(24.7010, device='cuda:0'), tensor(15.2156, device='cuda:0'), tensor(15.6530, device='cuda:0'), tensor(19.6549, device='cuda:0'), tensor(18.1315, device='cuda:0'), tensor(21.3067, device='cuda:0'), tensor(16.0705, device='cuda:0'), tensor(18.7581, device='cuda:0'), tensor(16.7534, device='cuda:0'), tensor(22.4920, device='cuda:0'), tensor(13.9861, device='cuda:0'), tensor(11.9104, device='cuda:0'), tensor(17.3335, device='cuda:0'), tensor(17.4368, device='cuda:0'), tensor(14.2797, device='cuda:0'), tensor(19.9819, device='cuda:0'), tensor(28.6680, device='cuda:0'), tensor(19.8752, device='cuda:0'), tensor(13.6943, device='cuda:0'), tensor(15.1842, device='cuda:0'), tensor(12.0210, device='cuda:0'), tensor(19.6827, device='cuda:0'), tensor(16.9316, device='cuda:0'), tensor(13.0888, device='cuda:0'), tensor(18.5715, device='cuda:0'), tensor(22.4879, device='cuda:0'), tensor(13.3826, device='cuda:0'), tensor(20.580